In [ ]:
import cv2
import pickle
import numpy as np
import pandas as pd
import mediapipe as mp
import argparse

# Load Best Model & Scaler
with open("./model/best_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Define Landmarks Used in Training
FEATURES = [
    "LEFT_SHOULDER", "RIGHT_SHOULDER", "LEFT_ELBOW", "RIGHT_ELBOW",
    "LEFT_WRIST", "RIGHT_WRIST"
]

# Function to Calculate Angle
def calculate_angle(a, b, c):
    """Calculate angle between three points."""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    return np.degrees(angle)

# Function to Process Video (Live or Uploaded)
def process_video(video_path=None):
    if video_path is None:
        cap = cv2.VideoCapture(0)  # Webcam Mode
    else:
        cap = cv2.VideoCapture(video_path)  # Uploaded Video Mode

    total_frames = 0
    correct_predictions = 0
    rep_count = 0
    last_label = None  # Track last movement phase (UP/DOWN)

    with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            total_frames += 1

            # Convert to RGB for Mediapipe
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                row = []

                # Extract Features Consistent with Training
                for lm in FEATURES:
                    keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                    row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

                # Convert Data & Predict
                X_input = pd.DataFrame([row], columns=[f"{lm.lower()}_{axis}" for lm in FEATURES for axis in ["x", "y", "z", "v"]])
                X_scaled = scaler.transform(X_input)
                prediction = model.predict(X_scaled)[0]

                label = "UP" if prediction == 0 else "DOWN"
                color = (0, 255, 0) if prediction == 0 else (0, 0, 255)

                # Accuracy Calculation
                if prediction == 0:
                    correct_predictions += 1

                # Calculate Elbow Angle for Rep Counting
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y]

                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y]

                left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

                # Classify Movement Phase & Count Reps
                elbow_threshold = 90  # Define elbow bend threshold
                if left_elbow_angle > elbow_threshold and right_elbow_angle > elbow_threshold:
                    current_label = "UP"
                else:
                    current_label = "DOWN"

                if last_label == "DOWN" and current_label == "UP":
                    rep_count += 1  # Count rep when moving from DOWN to UP
                
                last_label = current_label

                # Draw Pose on Image
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                # Display Prediction & Reps
                cv2.putText(image, f"Bench Press: {label}", (30, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
                cv2.putText(image, f"Reps: {rep_count}", (30, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)

            # Show Video Output
            cv2.imshow("Bench Press Detection", image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

    # Calculate Accuracy for Uploaded Video
    if video_path:
        accuracy = (correct_predictions / total_frames) * 100 if total_frames > 0 else 0
        print(f"Bench Press Accuracy: {accuracy:.2f} | Total Reps: {rep_count}")
        
        # Corrected formatting: Accuracy with % and Reps as a whole number
        with open("accuracy.txt", "w") as f:
            f.write(f"Accuracy: {accuracy:.2f}%\nTotal Reps: {rep_count}")


# Argument Parsing for CLI
parser = argparse.ArgumentParser(description="Live or Video Upload Bench Press Detection")
parser.add_argument("--video", type=str, help="Path to the video file (leave blank for live detection)")
args = parser.parse_args()

if args.video:
    process_video(args.video)
else:
    process_video()


FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pkl'